# ShellSage

> ShellSage saves sysadmins' sanity by solving shell script snafus super swiftly

ShellSage is an AI-powered command-line assistant that integrates seamlessly with your terminal workflow through tmux. It provides contextual help for shell operations, making it easier to navigate complex command-line tasks, debug scripts, and manage your system.

[![PyPI version](https://badge.fury.io/py/shell-sage.svg)](https://badge.fury.io/py/shell-sage)
[![Python 3.8+](https://img.shields.io/badge/python-3.10+-blue.svg)](https://www.python.org/downloads/)
[![License: MIT](https://img.shields.io/badge/License-MIT-yellow.svg)](https://opensource.org/licenses/MIT)
[![Documentation](https://img.shields.io/badge/docs-nbdev-blue.svg)](https://answerdotai.github.io/shell_sage/)

## Overview

**Core Features:**

- **AI-powered shell assistance** - Get instant help with commands, syntax, and system administration
- **Lisette integration** - Works with multiple LLM providers (Claude, GPT, Ollama, etc.)
- **Tmux-aware context** - Automatically reads your terminal history for contextual help
- **Tool integration** - Can view files, search code, create files, and make edits with your permission
- **Web search capability** - Search the internet for up-to-date information when needed
- **Piped input support** - Pipe command output or file contents directly to ShellSage
- **SQLite logging** - Save all interactions for later reference

**Tmux Workflow:**

- **Code block extraction** - Extract and send LLM-suggested commands directly to your tmux pane
- **Multi-pane support** - Can analyze context from all visible tmux panes

**Modes:**

- **Default mode** - Educational, friendly tone focused on teaching
- **Sassy mode** - GLaDOS-inspired personality with dry wit (still helpful!)

**Display:**

- **Rich markdown rendering** - Beautiful, syntax-highlighted output
- **Customizable themes** - Configure code highlighting to match your preferences

Whether you're a seasoned sadmin or just getting started with the command line, ShellSage acts as your intelligent terminal companion, ready to help with both simple commands and complex operations.

## Installation

Install ShellSage directly from PyPI using pip:

```sh
uv pip install shell-sage
```

### Prerequisites

1. **API Key Setup**
   ```sh
   # For Claude (default)
   export ANTHROPIC_API_KEY=sk...
   
   # For OpenAI (optional)
   export OPENAI_API_KEY=sk...
   ```
2. **tmux Configuration**
   
   I created a preconfigured [tmux configuration](.tmux.conf) that I found works well with shell sage. It enables things like mouse support, adds pane ids to your status bar so you can quickly reference them to have ShellSage read from them, turns off alternative-screen so editor content like vim will stay in the tmux buffer for ShellSage to see, and adds a shortcut for automatically extracting out code fence blocks into your command prompt (CTRL+B+E then the index of the code fence block you want).

## Getting Started

### Your First Command

Once installed, try ShellSage with a simple greeting:

```sh
ssage hi
```

If everything is properly setup, you should see a welcoming greeting back from ShellSage!

### Getting Help with Commands

The most basic use case is asking about shell commands:

```sh
ssage how do I list all files including hidden ones? # note if you are using zsh, you will need to wrap prompts in quotes that include ?
ssage explain what rsync does
ssage show me tar command examples
```

ShellSage will provide the command, explain how it works, and give you practical examples.

### Using Terminal Context

ShellSage automatically reads your tmux history to understand what you're working on:

```sh
# After running some commands that produced errors
ssage what went wrong with my last command?

# Get suggestions based on your workflow
ssage what should I do next?
```

### Piping Content for Analysis

One of ShellSage's most powerful features is analyzing piped input:

```sh
# Understand error messages
cat error.log | ssage explain this error

# Review code changes
git diff | ssage summarize these changes

# Analyze system logs
journalctl -xe | ssage what's causing these errors?
```

### Working with Multiple Tmux Panes

When you have multiple panes open, you can reference specific ones by their ID (shown in your status bar):

```sh
# Analyze what's happening in pane %2
ssage --pid %2 what is this process doing?

# Compare across all panes
ssage --pid all summarize what's happening in all my panes
```

### Extracting and Running Commands

When ShellSage suggests commands, you can extract them directly to your command line. First, enable logging:

```sh
ssage --log how do I find large files?
```

ShellSage will respond with code blocks. Press `Ctrl+B E` then enter the index number (0 for first block, 1 for second, etc.) to send that command directly to your prompt.

### Enabling Sassy Mode

For a more entertaining experience, try sassy mode (GLaDOS-inspired):

```sh
ssage --mode sassy explain git rebase
```

### Next Steps

Now that you're familiar with the basics:
- Try piping different types of content to ShellSage
- Experiment with the `--pid` flag to analyze different panes
- Enable logging with `--log` to use the code extraction feature
- Check out the Configuration section to customize your experience

## Configuration and Model Providers

### Configuration File

ShellSage can be customized through a configuration file located at `~/.config/shell_sage/shell_sage.conf`:

```
[DEFAULT]
model = 'claude-sonnet-4-5-20250929'    # Your preferred model
search = ''                             # Enable web search capability (can be either l,m,h https://lisette.answer.ai/#web-search)
mode = 'default'                        # or "sassy"
api_base = ''                           # alternative api url base
api_key = ''                            # alternative api key to use instead of default env var
history_lines = -1                      # Lines of terminal history to include. -1 means include all
code_theme = "monokai"                  # Syntax highlighting theme
code_lexer = "python"                   # Default lexer for inline code blocks
log = False                             # Enable SQLite logging (required for code extraction)
```

### Using Different Model Providers

ShellSage uses [lisette](https://github.com/AnswerDotAI/lisette) under the hood, which supports any LLM provider via [LiteLLM](https://docs.litellm.ai/docs/providers). This means you can use Claude, GPT, Gemini, local models via Ollama, and many others.

#### Local Models with Ollama

For privacy-conscious users or offline usage, run models locally:

```sh
# First, install and start Ollama, then pull a model
ollama pull qwen3:1.7b

# Use with ShellSage
ssage --model ollama_chat/qwen3:1.7b how do I compress a directory?

# Or set as default in your config.toml
model = "ollama_chat/qwen3:1.7b"
```

#### OpenAI

```sh
export OPENAI_API_KEY=your_key_here
ssage --model gpt-5 explain kubernetes pods
```

#### Google Gemini

```sh
export GEMINI_API_KEY=your_key_here
ssage --model gemini/gemini-pro what is systemd?
```

#### Other Providers

For any provider supported by LiteLLM, set the appropriate API key and use the provider's model format:

```sh
# Custom API base
ssage --api-base https://your-api.com --api-key your_key --model your_model your query
```

See the [LiteLLM providers documentation](https://docs.litellm.ai/docs/providers) for the complete list of supported providers and their model naming conventions.

### Command Line Overrides

Any configuration option can be temporarily overridden via command line arguments:

```sh
# Use a different model for one query
ssage --model gpt-5 explain this error

# Adjust history lines
ssage --history-lines 100 what commands did I just run?

# Enable logging for this prompt
ssage --log how do I find large files?

# Change the display theme
ssage --code-theme dracula --code-lexer python show me a python example
```

You can find all available code themes and lexers at https://pygments.org/styles/

## Contributing

ShellSage is built using [nbdev](https://nbdev.fast.ai/). For detailed contribution guidelines, please see our [CONTRIBUTING.md](CONTRIBUTING.md) file.

We welcome contributions of all kinds:

- Bug reports
- Feature requests
- Documentation improvements
- Code contributions

Please visit our [GitHub repository](https://github.com/AnswerDotAI/shell_sage) to get started.